<a href="https://colab.research.google.com/github/Buzon-coder/QRT-Electricity-price/blob/main/QRT_Electricity_PriceV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#test 2 pour github

In [ ]:
#from google.colab import files
#uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
X_train = pd.read_csv("X_train_NHkHMNU.csv")
y_train = pd.read_csv("y_train_ZAN5mwg.csv")
X_test = pd.read_csv("X_test_final.csv")

In [69]:
#print(X_train.head())
#print(X_train.shape)
#print(X_train.head())
#print(X_train.tail())
#print(X_train["ID"].nunique())
#print(X_train["DAY_ID"].nunique()) #--> ça return 851, donc il y a 851 DAY_ID uniques, donc il y a des jours qui apparaissent plusieurs fois
#print(X_train["ID"].isna().sum()) #--> ça return 0 donc tous les ID sont présents
#print(X_train['DAY_ID'].isna().sum()) #--> ça return 0 donc tous les jours sont présents
#print(X_train["COUNTRY"].nunique())
#print(X_train["COUNTRY"].isna().sum()) #--> la colonne COUNTRY est pleine
#print(X_train["ID"].min()) # --> ça return 0
#print(X_train["ID"].max()) # --> ça return 2146, je comprends pas très bien
#print(y_train.tail())
#print(y_train['TARGET'].isna().sum())
#print(X_test.shape)
#print(X_test.head())
#print(X_test.tail())
#print(X_test["ID"].nunique())
#print(X_test["DAY_ID"].min()) #--> ça return 4
#print(X_test["DAY_ID"].max()) #--> ça return 1214
#print(X_test["DAY_ID"].nunique()) #--> ça return 365, donc il faut prédire
#regarder comment faire pour voir s'il y a des jours (DAY_ID) qui apparaissent plus de deux fois

#je vais essayer de voir si j'ai des ID et des DAY_ID en commun entre X_TRAIN et X_TEST
#print(len(set(X_train["ID"])))
common_id = set(X_train["ID"]) & set(X_test["ID"])
print("Nombre d'ID' en commun entre TRAIN et TEST:",len(common_id))

common_days = set(X_train["DAY_ID"]) & set(X_test["DAY_ID"])
print("Nombre de jours en commun entre TRAIN et TEST:",len(common_days))

#print(len(set(X_test["DAY_ID"])))
#print(len(set(X_test["ID"])))

# ==> donc il n'y a aucun jour en commun et aucun ID en commun entre TRAIN et TEST
#dans mes données TRAIN, j'ai 1494 ID et 851 jours
#dans mes données TEST, j'ai 654 ID et 365 jours

#print(max(X_train["ID"].max(), X_test["ID"].max()))
#print(X_train["ID"].max())
#print(X_test["ID"].max())
#print(max(X_train["DAY_ID"].max(), X_test["DAY_ID"].max()))
#print(min(X_train["DAY_ID"]))
#print(max(X_train["DAY_ID"]))

#print(min(X_test["DAY_ID"]))
#print(max(X_test["DAY_ID"]))
#counts = X_train["DAY_ID"].value_counts()
#counts_more_than_2 = counts[counts>2]
#print(len(counts_more_than_2))

#counts_test = X_test["DAY_ID"].value_counts()
#counts_test_more_than_2 = counts_test[counts_test>2]
#print(len(counts_test_more_than_2))
print(X_test.head())

Nombre d'ID' en commun entre TRAIN et TEST: 0
Nombre de jours en commun entre TRAIN et TEST: 0
     ID  DAY_ID COUNTRY  DE_CONSUMPTION  FR_CONSUMPTION  DE_FR_EXCHANGE  \
0  1115     241      FR        0.340083       -0.433604       -0.423521   
1  1202    1214      FR        0.803209        0.780411        0.601610   
2  1194    1047      FR        0.795540        0.721954        1.179158   
3  1084    1139      FR        0.172555       -0.723427       -0.044539   
4  1135     842      FR        0.949714        0.420236        0.617391   

   FR_DE_EXCHANGE  DE_NET_EXPORT  FR_NET_EXPORT  DE_NET_IMPORT  ...  \
0        0.423521       0.165333       0.519419      -0.165333  ...   
1       -0.601610       0.342802       0.555367      -0.342802  ...   
2       -1.179158       1.620928       0.666901      -1.620928  ...   
3        0.044539            NaN      -0.205276            NaN  ...   
4       -0.617391       0.608561      -0.240856      -0.608561  ...   

   FR_RESIDUAL_LOAD   DE_RA

Donc il n'y a aucun jour en commun et aucun ID en commun entre TRAIN et TEST.
Dans mes données TRAIN, j'ai 1494 ID et 851 jours
Dans mes données TEST, j'ai 654 ID et 365 jours
J'ai donc un total de 2148 ID et 1216 jours.
C'est bien ce que je trouve lorsque je regarde l'ID max (je trouve 2147) et le DAY_ID max (je trouve 1215).
Les jours de mes données d'entraînement s'étalent de DAY_ID = 0 à DAY_ID = 1215, et les jours de mes données test s'étalent de DAY_ID = 4 à DAY_ID = 1214.
Donc l'objectif n'est pas de prédire les valeurs des futures (variation journalière du prix de contrats à terme sur l'électricité) dans le futur, mais en fonction des données météorologiques, énergétiques, ...

Aucun jour(=DAY_ID) dans X_train ou dans X_test n'apparaît plus de deux fois, donc dans mes données TRAIN, j'ai 643 jours qui apparaissent 2 fois, et 208 jours qui n'apparaissent qu'une fois, et dans mes données TEST, j'ai 289 jours qui apparaissent 2 fois, et 76 jours qui n'apparaissent qu'une fois.

In [79]:
# je trie X_train et y_train

X_train = X_train.sort_values(["DAY_ID", "COUNTRY"]).reset_index(drop=True)
y_train = y_train.set_index("ID").loc[X_train["ID"]].reset_index()
print(y_train["ID"])
print(X_train["ID"])
#y_train = y_train.sort_values(["DAY_ID", "COUNTRY"]).reset_index(drop=True)

#print(X_train.head())

0       1057
1       1179
2        111
3       1327
4        800
        ... 
1489     771
1490    1987
1491     472
1492    1688
1493    1049
Name: ID, Length: 1494, dtype: int64
0       1057
1       1179
2        111
3       1327
4        800
        ... 
1489     771
1490    1987
1491     472
1492    1688
1493    1049
Name: ID, Length: 1494, dtype: int64
